In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Project/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -r requirements.txt
!pip install sentencepiece
!pip install --upgrade pandas

  Using cached https://files.pythonhosted.org/packages/af/f3/683bf2547a3eaeec15b39cef86f61e921b3b187f250fcd2b5c5fb4386369/pandas-1.0.5-cp37-cp37m-manylinux1_x86_64.whl
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.0.5 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 6.0.4 which is incompatible.
  Found existing installation: pandas 1.2.2
    Uninstalling pandas-1.2.2:
      Successfully uninstalled pandas-1.2.2


  Using cached https://files.pythonhosted.org/packages/4c/33/87b15a5baeeb71bd677da3579f907e97476c5247c0e56a37079843af5424/pandas-1.2.2-cp37-cp37m-manylinux1_x86_64.whl
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 6.0.4 which is incompatible.
  Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5


In [ ]:
import os
import pathlib
import torch
from DeBERTa.DeBERTa.deberta import pretrained_models, tokenizers
from deberta import DeBERTaReconfig
import re
import string

file_path = pathlib.Path.cwd()

pretrained_model_base = {
    'model_class': pretrained_models['base'],
    'model_path': file_path.joinpath('model', 'base', 'pytorch_model.bin'),
    'model_config_path': file_path.joinpath('model', 'base', 'config.json'),
    'model_vocab_path': file_path.joinpath('model', 'base', 'bpe_encoder.bin'),
    'model_vocab_type': 'gpt2'
}

class DeBERTaTxtClassifier(torch.nn.Module):
    def __init__(self, model_path, model_config_path, freeze_deberta=True):
        super().__init__()
        self.deberta = DeBERTaReconfig(model_path, model_config_path)
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate an one-layer feed-forward classifier
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(D_in, H),
            torch.nn.ReLU(),
            #torch.nn.Dropout(0.5),
            torch.nn.Linear(H, D_out)
        )

        if freeze_deberta:
            for param in self.deberta.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):

        outputs = self.deberta(input_ids=input_ids,
                            attention_mask=attention_mask, output_all_encoded_layers=False)
        nextTensor = torch.empty(len(input_ids), 768, dtype=torch.float)
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0]
        
        for i in range(len(last_hidden_state_cls)):
            nextTensor[i] = last_hidden_state_cls[i][0]
        # Feed input to classifier to compute logits
        logits = self.classifier(nextTensor)

        return logits

#setting model here. Change pretrained_model between the base and v2xxlarge
pretrained_model = pretrained_model_base
max_seq_len = 512
vocab_path = pretrained_model['model_vocab_path']
vocab_type = pretrained_model['model_vocab_type']


In [ ]:
import numpy as np # linear algebra
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#data preprocessing
ps = PorterStemmer()
lem = WordNetLemmatizer()

def clean_text(text):
        text = re.sub('[^a-zA-Z0-9]',' ', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
        text = text.lower()
        return text

def text_process(text):
    text = text.split()
    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    return text

def preprocess(filepath, basepath = './datasets/raw/'):
    df = pd.read_csv(basepath + filepath)
    df = df.dropna()
    if (filepath == 'fake/train.csv'):
        X = df.drop('label',axis=1)
        y = df['label']
        X = X.text
        #X = fake_X.apply(clean_text).apply(text_process)
    if (filepath == 'ag.csv'):
        print(df)
  
    return X, y

fake_X, fake_y = preprocess('fake/train.csv')
print(fake_X)

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20795    Rapper T. I. unloaded on black celebrities who...
20796    When the Green Bay Packers lost to the Washing...
20797    The Macy’s of today grew from the union of sev...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799      David Swanson is an author, activist, journa...
Name: text, Length: 18285, dtype: object


In [ ]:
def tokenize(input_sentences, max_seq_len = 512):
    tokenized_sentences = {}
    masks = {}
    for key, sentence in input_sentences.items():
        tokenizer = tokenizers[vocab_type](vocab_path)
        tokens = tokenizer.tokenize(sentence)
        tokens = tokens[:max_seq_len - 2]
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1]*len(input_ids)
        paddings = max_seq_len-len(input_ids)
        input_ids = input_ids + [0]*paddings
        input_mask = input_mask + [0]*paddings
        tokenized_sentences[key] = input_ids
        masks[key] = input_mask
    return tokenized_sentences, masks

fake_tokenized, fake_masks = tokenize(fake_X)


In [ ]:
import time
import collections
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler



def train(model, X_train, X_masks, Y_train):
    X_train = collections.OrderedDict(sorted(X_train.items()))
    xlist = []
    ylist = []
    masklist = []
    for key, val in X_train.items():
        xlist.append(val)
        ylist.append(Y_train[key])
        masklist.append(X_masks[key])
    train_inputs = torch.tensor(xlist)
    train_masks = torch.tensor(masklist)
    train_labels = torch.tensor(ylist)
    batch_size = 32
    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    start = time.time()
    optimizer = torch.optim.Adam(model.parameters(),
                      lr=5e-5  
                      )
    epochs = 4
    total_steps = len(train_dataloader) * epochs
    loss_fn = torch.nn.CrossEntropyLoss();

    for step, batch in enumerate(train_dataloader):
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t for t in batch)
        model.zero_grad()
        y_pred = model(b_input_ids, b_attn_mask)
        loss = loss_fn(y_pred, b_labels)
        loss.backward()
        optimizer.step()
    print('Finished training model in %.1f sec' % ((time.time()-start)))


In [ ]:
fake_model = DeBERTaTxtClassifier(pretrained_model['model_path'], pretrained_model['model_config_path'])
train(fake_model, fake_tokenized, fake_masks, fake_y)

needs to be (state_dict post BERT config): 244
is (model state): 203
Finished training model in 6260.1 sec
